<a href="https://colab.research.google.com/github/RamzNN/RamzNN/blob/main/filter_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
df = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Работа/Платформа-парсер "Сапфир"/stage_one/выгрузка_вк.xlsx')

In [3]:
df.tail()

,group_id,name,description,Subscribers Count,relevance,Threat assesment status,city,country,tags,subcultures
54278,56230843,?? ФК Спартак Москва ?? Красно-белый НАВСЕГДА!,NaN,4699,True,destructive,Москва,Россия,ФК Спартак,Группы футбольных хулиганов
54279,62454446,Ролевая фурри,NaN,6024,True,destructive,NaN,Россия,Фурри,Антисемейные ценности
54280,60343205,НОВИЙ ПОРЯДОК,NaN,11818,True,destructive,Киев,Украина,Український порядок,Политический экстремизм
54281,56146201,Великі Вуйки/Останній москаль,NaN,23295,True,destructive,Киев,Украина,Москаль,Политический экстремизм
54282,55284725,Команда Навального,NaN,205372,True,destructive,NaN,NaN,Навальный,Политический экстремизм


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54283 entries, 0 to 54282
Data columns (total 10 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   group_id                 53583 non-null  object
 1   name                     54280 non-null  object
 2   description              40436 non-null  object
 3   Subscribers Count        53887 non-null  object
 4   relevance                54283 non-null  bool  
 5   Threat assesment status  53968 non-null  object
 6   city                     51682 non-null  object
 7   country                  52236 non-null  object
 8   tags                     54283 non-null  object
 9   subcultures              54283 non-null  object
dtypes: bool(1), object(9)
memory usage: 3.8+ MB


In [11]:
# df.drop_duplicates(subset=['name', 'description', 'relevance'], inplace=True)
# df.info()

In [5]:
# Проверка пропусков
print("\nКоличество пропусков по столбцам:")
print(df.isnull().sum())

# Распределение целевой переменной 'relevance'
print("\nРаспределение 'relevance':")
print(df['relevance'].value_counts())


Количество пропусков по столбцам:
group_id                     700
name                           3
description                13847
Subscribers Count            396
relevance                      0
Threat assesment status      315
city                        2601
country                     2047
tags                           0
subcultures                    0
dtype: int64

Распределение 'relevance':
relevance
False    45142
True      9141
Name: count, dtype: int64


In [6]:
# Если name пустое — заполним пустой строкой
df['name'] = df['name'].fillna('')
# Если description пустое — заполним пустой строкой
df['description'] = df['description'].fillna('')

In [7]:
# Проверка пропусков
print("\nКоличество пропусков по столбцам:")
print(df.isnull().sum())

# Распределение целевой переменной 'relevance'
print("\nРаспределение 'relevance':")
print(df['relevance'].value_counts())


Количество пропусков по столбцам:
group_id                    700
name                          0
description                   0
Subscribers Count           396
relevance                     0
Threat assesment status     315
city                       2601
country                    2047
tags                          0
subcultures                   0
dtype: int64

Распределение 'relevance':
relevance
False    45142
True      9141
Name: count, dtype: int64


In [8]:
# Объединение названия и описания
df['text'] = df['name'].astype(str) + ' ' + df['description'].astype(str)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54283 entries, 0 to 54282
Data columns (total 11 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   group_id                 53583 non-null  object
 1   name                     54283 non-null  object
 2   description              54283 non-null  object
 3   Subscribers Count        53887 non-null  object
 4   relevance                54283 non-null  bool  
 5   Threat assesment status  53968 non-null  object
 6   city                     51682 non-null  object
 7   country                  52236 non-null  object
 8   tags                     54283 non-null  object
 9   subcultures              54283 non-null  object
 10  text                     54283 non-null  object
dtypes: bool(1), object(10)
memory usage: 4.2+ MB


### Обработка текста

В этот раз я попробую без очистки и обработки текста но потом надо будет это сделать и сравнить качество

```Python
def clean_text(text):
    # Приводим к нижнему регистру
    text = text.lower()
    # Удаляем нежелательные символы (можно кастомизировать)
    text = re.sub(r'[^а-яa-z0-9ё\s.,!?–-]', ' ', text)
    # Убираем лишние пробелы
    text = re.sub(r'\s+', ' ', text).strip()
    return text

df['text'] = df['text'].apply(clean_text)

```

In [10]:
print(df['relevance'].value_counts())

relevance
False    45142
True      9141
Name: count, dtype: int64


In [11]:
df['text_length'] = df['text'].apply(lambda x: len(x.split()))
df['text_length'].describe()

,text_length
count,54283.000000
mean,65.324061
std,112.336866
min,1.000000
25%,4.000000
50%,18.000000
75%,71.000000
max,734.000000


In [12]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df, test_size=0.1, random_state=42, stratify=df['relevance'])
train_df, val_df = train_test_split(train_df, test_size=0.1, random_state=42, stratify=train_df['relevance'])

### Установка нужных библиотек

In [13]:
!pip install --upgrade pip
!pip install transformers accelerate evaluate torch sentencepiece

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 30.5 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 169.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 180.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 46.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 69.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 69.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 63.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 M

In [14]:
import pandas as pd
import numpy as np
import torch

from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import evaluate

# Убедимся, что у нас есть CUDA (если доступно)
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

# Предположим, вы уже прочитали df, train_df, val_df, test_df
# как мы делали ранее

Using device: cuda


In [15]:
train_df['relevance'] = train_df['relevance'].astype(int)
val_df['relevance'] = val_df['relevance'].astype(int)
test_df['relevance'] = test_df['relevance'].astype(int)

In [17]:
model_name = "ai-forever/ruBert-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=2  # бинарная классификация
)
model.to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ai-forever/ruBert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(120138, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1

### Подготовка данных для Trainer

In [25]:
MAX_LEN = 256  # Так как средняя длина ~65 слов, но max ~734, выберем 512

def tokenize_function(examples):
    # examples – это dict с ключами "text" и "relevance"
    return tokenizer(
        examples["text"],
        truncation=True,
        max_length=MAX_LEN,
        padding="max_length",  # или "longest", но max_length=512 безопаснее для batched training
    )


In [26]:
# Для Trainer нужно, чтобы в словаре для каждой строчки были хотя бы поля: 'text' и 'labels' (или 'label').

train_data = {
    "text": train_df["text"].tolist(),
    "labels": train_df["relevance"].tolist()
}

val_data = {
    "text": val_df["text"].tolist(),
    "labels": val_df["relevance"].tolist()
}

test_data = {
    "text": test_df["text"].tolist(),
    "labels": test_df["relevance"].tolist()
}


In [27]:
from datasets import Dataset

train_dataset = Dataset.from_dict(train_data)
val_dataset = Dataset.from_dict(val_data)
test_dataset = Dataset.from_dict(test_data)

# Применяем токенизацию
train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

# Фильтруем ненужные колонки и переименовываем labels -> label (или наоборот)
train_dataset = train_dataset.remove_columns(["text"])
val_dataset = val_dataset.remove_columns(["text"])
test_dataset = test_dataset.remove_columns(["text"])
# Теперь в каждом датасете есть "input_ids", "attention_mask", (возможно "token_type_ids") и "labels".


Map:   0%|          | 0/43968 [00:00<?, ? examples/s]

Map:   0%|          | 0/4886 [00:00<?, ? examples/s]

Map:   0%|          | 0/5429 [00:00<?, ? examples/s]

### Настройка Trainer

In [28]:
accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    acc = accuracy_metric.compute(predictions=predictions, references=labels)
    f1 = f1_metric.compute(predictions=predictions, references=labels, average="binary")

    return {
        "accuracy": acc["accuracy"],
        "f1": f1["f1"]
    }

In [29]:
batch_size = 8  # Можно увеличить, если хватает GPU (16/32).
num_epochs = 3  # Чаще всего 3-5 эпох хватает для fine-tuning
learning_rate = 1e-5

training_args = TrainingArguments(
    output_dir="my_sbert_model",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=learning_rate,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="f1",  # Следим за f1
    greater_is_better=True,
    save_total_limit=2,
    logging_steps=100,               # Логировать каждые 100 шагов
    logging_dir="./logs",            # Директория для логов (если нужна)
    report_to="none",                # Чтобы логи шли просто в консоль
    disable_tqdm=False               # Разрешить tqdm-прогресс‐бар
    # FP16 ?
    # fp16=True,  # Если GPU поддерживает half precision, можно ускорить
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [31]:
from transformers import EarlyStoppingCallback

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,  # Чтобы Trainer мог делать pad/collate
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=1)]
)

<ipython-input-31-e6fa9a6f883a>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [32]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.343500,0.366878,0.876995,0.524901


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.343500,0.366878,0.876995,0.524901
2,0.234700,0.391054,0.878019,0.600536
3,0.244000,0.471649,0.879656,0.595041


TrainOutput(global_step=16488, training_loss=0.3081199097899428, metrics={'train_runtime': 7026.2683, 'train_samples_per_second': 18.773, 'train_steps_per_second': 2.347, 'total_flos': 1.735270032310272e+16, 'train_loss': 0.3081199097899428, 'epoch': 3.0})

In [33]:
trainer.save_model("my_sbert_model/final")
tokenizer.save_pretrained("my_sbert_model/final")

('my_sbert_model/final/tokenizer_config.json',
 'my_sbert_model/final/special_tokens_map.json',
 'my_sbert_model/final/vocab.txt',
 'my_sbert_model/final/added_tokens.json',
 'my_sbert_model/final/tokenizer.json')

In [34]:
model.save_pretrained('/content/drive/MyDrive/Colab Notebooks/Работа/Платформа-парсер "Сапфир"/stage_one/my_sbert_model/final')
tokenizer.save_pretrained('/content/drive/MyDrive/Colab Notebooks/Работа/Платформа-парсер "Сапфир"/stage_one/my_sbert_model/final')


('/content/drive/MyDrive/Colab Notebooks/Работа/Платформа-парсер "Сапфир"/stage_one/my_sbert_model/final/tokenizer_config.json',
 '/content/drive/MyDrive/Colab Notebooks/Работа/Платформа-парсер "Сапфир"/stage_one/my_sbert_model/final/special_tokens_map.json',
 '/content/drive/MyDrive/Colab Notebooks/Работа/Платформа-парсер "Сапфир"/stage_one/my_sbert_model/final/vocab.txt',
 '/content/drive/MyDrive/Colab Notebooks/Работа/Платформа-парсер "Сапфир"/stage_one/my_sbert_model/final/added_tokens.json',
 '/content/drive/MyDrive/Colab Notebooks/Работа/Платформа-парсер "Сапфир"/stage_one/my_sbert_model/final/tokenizer.json')

### Оценка на тесте

In [35]:
test_metrics = trainer.evaluate(test_dataset)
print(test_metrics)

{'eval_loss': 0.38629522919654846, 'eval_accuracy': 0.8810093939952109, 'eval_f1': 0.6075334143377886, 'eval_runtime': 82.6311, 'eval_samples_per_second': 65.702, 'eval_steps_per_second': 8.217, 'epoch': 3.0}


In [36]:
predictions = trainer.predict(test_dataset)
preds = np.argmax(predictions.predictions, axis=1)
labels = predictions.label_ids

# Подсчет метрик вручную, если нужно:
acc = accuracy_metric.compute(predictions=preds, references=labels)
f1 = f1_metric.compute(predictions=preds, references=labels)
print("Accuracy:", acc["accuracy"])
print("F1:", f1["f1"])


Accuracy: 0.8810093939952109
F1: 0.6075334143377886


### Подбор порога фильтрации

#### Получаем вероятности на валидационном (или тестовом) наборе

In [37]:
import numpy as np
from sklearn.metrics import recall_score

# Предположим, val_dataset (или test_dataset) — ваши данные для подбора порога
val_result = trainer.predict(val_dataset)
logits = val_result.predictions  # shape (num_samples, 2) для бинарной классификации
labels = val_result.label_ids    # истина (0 или 1)

# Превращаем логиты в вероятности для класса "True" (индекс 1)
import torch
probs = torch.nn.functional.softmax(torch.tensor(logits), dim=-1).numpy()
probs_true = probs[:, 1]  # вероятность принадлежности к классу True


In [49]:
thresholds = np.linspace(0, 1, 101)  # 0.00, 0.01, 0.02, ..., 1.00

best_threshold = None
best_diff = float('inf')  # чтобы минимизировать |recall_false - 0.40|
best_recall_false = None
best_recall_true = None

for thr in thresholds:
    # Бинарные предсказания по выбранному порогу
    pred = (probs_true >= thr).astype(int)  # 1, если prob >= thr

    # Считаем recall для каждого класса
    recs = recall_score(labels, pred, average=None)
    recall_false = recs[0]  # для класса 0
    recall_true  = recs[1]  # для класса 1

    # Проверяем условие recall_true >= 0.97
    if recall_true >= 0.90:
        # Смотрим, насколько recall_false близок к 0.40
        diff = abs(recall_false - 0.50)
        if diff < best_diff:
            best_diff = diff
            best_threshold = thr
            best_recall_false = recall_false
            best_recall_true = recall_true

# Проверяем, нашли ли мы подходящий порог
if best_threshold is not None:
    print(f"Подходящий порог: {best_threshold:.2f}")
    print(f"Recall False = {best_recall_false:.3f} (хотели около 0.40)")
    print(f"Recall True  = {best_recall_true:.3f} (>= 0.97)")
    print(f"Разница от 0.40 по False: {best_diff:.3f}")
else:
    print("Не удалось найти порог, при котором recall_true >= 0.97 и recall_false ~ 0.40.")


Подходящий порог: 0.01
Recall False = 0.579 (хотели около 0.40)
Recall True  = 0.902 (>= 0.97)
Разница от 0.40 по False: 0.079


### Эксперимент

Вариант 3: Подбирать порог иначе (двухшаговый подход)

Иногда бизнес-логика звучит не «хотим recallfalse=0.4recallfalse​=0.4», а «хотим убрать 40% объектов из всего датасета, которые модель считает наиболее «подозрительными» на False». Тогда мы можем:

    Отсортировать все объекты по вероятности класса True (по возрастанию).
    Взять нижние 40% объектов (т.е. с наименьшей вероятностью True) и объявить их False. Остальных (60%) объявить True.
    Посчитать, какой recalltruerecalltrue​ получится. Если он < 0.97, значит модель не умеет так хорошо различать.

In [51]:
import numpy as np
import torch
from sklearn.metrics import recall_score

# 1. Получаем логиты и метки на валидационной / тестовой выборке:
val_result = trainer.predict(val_dataset)
logits = val_result.predictions         # shape: (num_samples, 2)
labels = val_result.label_ids           # истинные классы 0 или 1

# 2. Преобразуем логиты в вероятности принадлежности к классу "True" (индекс=1):
probs = torch.nn.functional.softmax(torch.tensor(logits), dim=-1).numpy()
probs_true = probs[:, 1]  # вероятность класса True

# 3. Сортируем индексы выборки по возрастанию вероятности True (самые "False" - первые):
idx_sorted = np.argsort(probs_true)
n = len(probs_true)

# 4. Находим, сколько объектов = 40% от всего датасета:
cutoff = int(0.35 * n)  # нижние 40%

# 5. Формируем кастомные предсказания:
#    - те, у кого prob True минимальна (попадают в нижние 40%) -> ставим класс 0 (False)
#    - остальные 60% -> класс 1 (True)
preds_custom = np.zeros(n, dtype=int)   # по умолчанию 0 (False)
top_idx = idx_sorted[cutoff:]           # индексы, которые идут в верхние 60%
preds_custom[top_idx] = 1

# 6. Считаем Recall по отдельным классам
#    average=None -> вернет [recall_for_class_0, recall_for_class_1]
rec_each = recall_score(labels, preds_custom, average=None)
recall_false = rec_each[0]
recall_true = rec_each[1]

print(f"Всего объектов: {n}")
print(f"Убрали (предсказали False) в итоге: {cutoff} ({cutoff/n:.2%} от датасета)")
print(f"Recall(False) = {recall_false:.3f}")
print(f"Recall(True)  = {recall_true:.3f}")


Всего объектов: 4886
Убрали (предсказали False) в итоге: 1710 (35.00% от датасета)
Recall(False) = 0.410
Recall(True)  = 0.948


#### Интерпретация результатов

    cutoff — фактически определяет, сколько объектов мы отбрасываем. Так как это фиксированные 40%, модель не выбирает порог по recall, а вы жёстко задаёте «хочу убрать ровно 40%».
    recall_true показывает, какую долю реальных True мы сохранили (среди оставшихся 60%). Если получается 0.95 или 0.90 — значит “потеряли” 5% или 10% True.
    recall_false показывает, какую часть реальных False мы убрали (если 0.80, значит мы отсекаем 80% всех False, а 20% False остаются).

Важно: этот метод управления работает по принципу “сначала выбираем долю (40%) для удаления, а уже потом смотрим, что там оказалось”. Если ваша реальная цель — убрать минимум 40% мусора и при этом терять не более 3% True, и выяснилось, что при отбрасывании ровно 40% теряется 10% True → модель не справляется с таким требованием.




- Общее число объектов: 4886
- Удалено (предсказано как False):ровно 35% выборки.
- Recall(False) = 0.410 означает, что среди всех действительно ложных сообществ вы «споймали» (и удалили) 41%.
- Recall(True) = 0.948 означает, что среди всех действительно релевантных сообществ вы сохранили 94.8% (то есть потеряли ~5.2% истинно релевантных).

Как интерпретировать на практике

- Вы “убираете” 35% датасета, которые выглядят наименее вероятными к классу True (релевантные).
- Из реального «мусора» (False) в результате отсеивается почти половина (41%).
- При этом вы случайно выбрасываете около 5.2% настоящих (True) сообществ.

Если вам подходит такой компромисс – убрать 40% мусора ценой 5% потерь релевантных сообществ, – то да, вы можете внедрить такой фильтр в сайт.